In [6]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
from google.colab import drive
drive.mount('/content/drive')

# Load data from CSV
data = pd.read_csv('/content/drive/MyDrive/amazon_data_refined1.csv', error_bad_lines=False, low_memory=False)

# Data preprocessing
stemmer = SnowballStemmer("english")
stopword = set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

data['Combined_Text'] = data['Reviews'].astype(str)
text_data = data['Combined_Text'].apply(lambda x: str(x)).apply(clean)

Mounted at /content/drive


<ipython-input-7-0ec6ac3781cd>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('/content/drive/MyDrive/amazon_data_refined1.csv', error_bad_lines=False, low_memory=False)


In [8]:
# Convert sentiment labels to numerical format
label_mapping = {'Positive': 2, 'Neutral': 1, 'Negative': 0}
data['Output'] = data['Output'].map(label_mapping)
labels = data['Output'].values

# Tokenize text data
max_words = 10000
max_seq_length = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_data)
sequences = tokenizer.texts_to_sequences(text_data)
X = pad_sequences(sequences, maxlen=max_seq_length)

# Convert sentiment labels to one-hot encoding
num_classes = 3
y = to_categorical(labels, num_classes=num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model architecture using RNN
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_seq_length))
model.add(SimpleRNN(128))  # You can adjust the number of units here
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Epoch 1/5
230/230 [==============================] - 23s 91ms/step - loss: 0.6182 - accuracy: 0.7477 - val_loss: 0.4140 - val_accuracy: 0.8468
Epoch 2/5
230/230 [==============================] - 22s 95ms/step - loss: 0.2250 - accuracy: 0.9286 - val_loss: 0.2815 - val_accuracy: 0.9064
Epoch 3/5
230/230 [==============================] - 21s 91ms/step - loss: 0.0926 - accuracy: 0.9735 - val_loss: 0.3165 - val_accuracy: 0.9078
Epoch 4/5
230/230 [==============================] - 22s 96ms/step - loss: 0.0589 - accuracy: 0.9842 - val_loss: 0.3201 - val_accuracy: 0.9119
Epoch 5/5
115/115 [==============================] - 2s 14ms/step - loss: 0.3630 - accuracy: 0.9026
Test Loss: 0.3629966378211975
Test Accuracy: 0.9026115536689758
